In [1]:
!pip install gluonnlp pandas tqdm   
!pip install mxnet
!pip install sentencepiece
!pip install transformers
!pip install torch
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl size=681108 sha256=d78402edda356caa9ed31f04ed53138d7d4acf1045d98d10e7788e3899db3aa8
  Stored in directory: /root/.cache/pip/wheels/1a/1e/0d/99f55911d90f2b95b9f7c176d5813ef3622894a4b30fde6bd3
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 19.3 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip uninstall -y gluonnlp
!pip install gluonnlp==0.9.1

Found existing installation: gluonnlp 0.10.0
Uninstalling gluonnlp-0.10.0:
  Successfully uninstalled gluonnlp-0.10.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.8/252.8 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.9.1-cp310-cp310-linux_x86_64.whl size=557742 sha256=ff95b3ad30114a26a569f02107fb53b84195af0ad3e3283804c60fda91ce3d6e
  Stored in directory: /root/.cache/pip/wheels/fc/5b/9c/3295bb07f7c5544a96303a48988707816f44a536e8e1413922
Successfully built gluonnlp


In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd
from sklearn.model_selection import train_test_split
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from google.colab import drive
import gluonnlp as nlp

In [4]:
drive.mount('/content/drive')
device = torch.device("cuda:0")

Mounted at /content/drive


In [5]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')
tok = tokenizer.tokenize

# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [6]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):
   
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(float(i[label_idx])) for i in dataset]

    def __getitem__(self, i):
        return self.sentences[i] + (self.labels[i],)
         
    def __len__(self):
        return len(self.labels)

In [7]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=8, # keywords 라벨 개수
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [8]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
def predict(sentence):
    dataset = [[sentence, '0']]
    test = BERTDataset(dataset, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(test, batch_size=batch_size, num_workers=2)
    model.eval()
    answer = 0
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        for logits in out:
            logits = logits.detach().cpu().numpy()
            answer = np.argmax(logits)
    return answer

In [9]:
%cd './drive/MyDrive/Colab Notebooks/Github/ufriends-chatbot/'

/content/drive/MyDrive/Colab Notebooks/Github/ufriends-chatbot


In [10]:
!pwd

/content/drive/MyDrive/Colab Notebooks/Github/ufriends-chatbot


In [11]:
train_df = pd.read_csv('Dataset/Preprocessing/train_df_preprocess.csv')
validation_df = pd.read_csv('Dataset/Preprocessing/validation_df_preprocess.csv')

In [12]:
train_df

,keywords,major_emotions,Q,A
0,2.0,5.0,아내가 드디어 출산하게 되어서 정말 신이 나,아내분이 출산을 하시는군요 정말 축하드려요
1,3.0,2.0,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야,약 종류가 많아 번거로우시겠어요
2,5.0,4.0,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워,고등학교 수업이 중학교 때와는 다르게 갑자기 어려워서 당황스러우시군요
3,4.0,5.0,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야 너무 행복해,재취업 후 첫 월급이라 정말 기쁘시겠어요
4,4.0,5.0,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어,기분 좋으시겠어요 앞으로는 어떻게 하실 생각이신가요
...,...,...,...,...
175139,3.0,2.0,미리 미리 건강 챙기고 모두 안 아팠으면 좋겠어,친구들과의 전화로 기분이 나아졌으면 좋겠어요
175140,4.0,4.0,주변에 믿음직한 사람들에게서 정보도 많이 얻고 달콤한 말은 항상 의심하고 볼래,신뢰가 있는 사람들에게서 정보도 얻고 하고자 하는 일도 잘되길 바라요
175141,3.0,2.0,친구들에게 내 마음을 터놓고 얘기하면 좀 나아질 것 같아,친구들과 술 한잔하시고 기분이 좋아지셨으면 좋겠어요
175142,0.0,4.0,남편에게 이런 내 마음을 솔직하게 얘기해 봐야겠어,솔직하게 얘기하셔서 외롭고 우울한 마음이 나아지셨으면 좋겠어요


In [13]:
train_set = train_df.loc[:, ['keywords', 'Q']]
validation_set = validation_df.loc[:, ['keywords', 'Q']]

In [14]:
train_set.dropna(inplace=True)
validation_set.dropna(inplace=True)

In [15]:
train_set_data = [[i, str(j)] for i, j in zip(train_set['Q'], train_set['keywords'])]
validation_set_data = [[i, str(j)] for i, j in zip(validation_set['Q'], validation_set['keywords'])]

train_set_data, test_set_data = train_test_split(train_set_data, test_size = 0.2, random_state=4)

train_set_data = BERTDataset(train_set_data, 0, 1, tok, vocab, max_len, True, False)
test_set_data = BERTDataset(test_set_data, 0, 1, tok, vocab, max_len, True, False)
train_dataloader = torch.utils.data.DataLoader(train_set_data, batch_size=batch_size, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(test_set_data, batch_size=batch_size, num_workers=2)

In [16]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

# epoch을 5로 설정
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0

    # train set
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    # test set
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
<ipython-input-16-daa5795bd9ea>:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/2190 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 2.120027542114258 train acc 0.0625
epoch 1 batch id 201 loss 1.9307321310043335 train acc 0.20942164179104478
epoch 1 batch id 401 loss 1.2753227949142456 train acc 0.3341256234413965
epoch 1 batch id 601 loss 1.3561736345291138 train acc 0.4122816139767055
epoch 1 batch id 801 loss 1.2442179918289185 train acc 0.4564021535580524
epoch 1 batch id 1001 loss 1.21511971950531 train acc 0.48284527972027974
epoch 1 batch id 1201 loss 1.3567055463790894 train acc 0.5005334096586178
epoch 1 batch id 1401 loss 0.8288720846176147 train acc 0.5145989471805853
epoch 1 batch id 1601 loss 1.0675550699234009 train acc 0.5250722204871955
epoch 1 batch id 1801 loss 1.0726985931396484 train acc 0.534139019988895
epoch 1 batch id 2001 loss 0.9659137725830078 train acc 0.5420180534732634
epoch 1 train acc 0.5472645547945206


<ipython-input-16-daa5795bd9ea>:41: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/548 [00:00<?, ?it/s]

...

epoch 4 test acc 0.6357835310218979


  0%|          | 0/2190 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.5118855237960815 train acc 0.84375
epoch 5 batch id 201 loss 0.4731064438819885 train acc 0.7690453980099502
epoch 5 batch id 401 loss 0.6250147223472595 train acc 0.7676122194513716
epoch 5 batch id 601 loss 0.8610291481018066 train acc 0.7671589018302829
epoch 5 batch id 801 loss 0.6813979744911194 train acc 0.7715941011235955
epoch 5 batch id 1001 loss 0.5448724031448364 train acc 0.772977022977023
epoch 5 batch id 1201 loss 0.6338707208633423 train acc 0.7757207535387177
epoch 5 batch id 1401 loss 0.4146609604358673 train acc 0.7772350107066381
epoch 5 batch id 1601 loss 0.48305246233940125 train acc 0.778829637726421
epoch 5 batch id 1801 loss 0.6427874565124512 train acc 0.7798011521377013
epoch 5 batch id 2001 loss 0.45119863748550415 train acc 0.780250499750125
epoch 5 train acc 0.7805270167427703


  0%|          | 0/548 [00:00<?, ?it/s]

epoch 5 test acc 0.6345346715328467


In [17]:
!pwd

/content/drive/MyDrive/Colab Notebooks/Github/ufriends-chatbot


In [18]:
torch.save(model, f'Model/keywords-train-SentimentAnalysisKOBert.pt')
torch.save(model.state_dict(), f'Model/keywords-train-SentimentAnalysisKOBert_StateDict.pt')

In [ ]:
# 대인관계, 대인관계(부부, 자녀) - 0 
# 진로, 취업, 직장, 업무 스트레스 - 1
# 연애, 결혼, 출산 - 2
# 건강, 죽음 - 3
# 재정, 은퇴, 노후준비 - 4
# 학업 및 진로 - 5
# 학교폭력, 따돌림 - 6
# 가족관계 - 7

In [19]:
sentence = '친구랑 싸워서 속상해'
predict(sentence)

0

In [20]:
sentence = '회사에서 해고 당했어'
predict(sentence)

1

In [21]:
sentence = '아이를 키우려면 돈이 많이 들어'
predict(sentence)

2

In [22]:
sentence = '비타민 챙겨 먹어야지'
predict(sentence)

3

In [23]:
sentence = '자식들과 여행하며 살고싶어'
predict(sentence)

4

In [24]:
sentence = '시험에서 떨어지면 어떡하지?'
predict(sentence)

5

In [25]:
sentence = '친구들을 괴롭히는 것은 나빠'
predict(sentence)

6

In [26]:
sentence = '부모님과 싸웠어'
predict(sentence)

7